In [0]:
# Install the ODBC driver
%pip install pyodbc

In [0]:
dbutils.library.restartPython()

In [0]:
%sh
# Import Microsoft GPG key
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -

# Add Microsoft package repository
curl https://packages.microsoft.com/config/ubuntu/20.04/prod.list > /etc/apt/sources.list.d/mssql-release.list

# Update package lists
apt-get update

# Install ODBC Driver 18 for SQL Server
ACCEPT_EULA=Y apt-get install -y msodbcsql18


In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, BooleanType
import logging
import pyodbc
import ast
import os
import uuid
from datetime import datetime

# Set up logging
log_dir = "/dbfs/tmp/logs"  # Separate directory for logs
os.makedirs(log_dir, exist_ok=True)
log_file = os.path.join(log_dir, f"myapp_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")

logging.basicConfig(
    filename=log_file,
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Get metadata and image path from Databricks widgets
# Create widgets for metadata and image data path
dbutils.widgets.text("metadata", "")
dbutils.widgets.text("imageDataPath", "")

# Retrieve widget values
metadata_str = dbutils.widgets.get("metadata")
image_data_path = dbutils.widgets.get("imageDataPath")

# Log retrieved values
logger.info(f"Received metadata: {metadata_str}")
logger.info(f"Received image data path: {image_data_path}")


# Parse the metadata string into a dictionary
metadata = ast.literal_eval(metadata_str)

# Azure SQL Database connection settings
server   = dbutils.secrets.get(scope="dear-keyvault-scope", key="dearsql-server")
database = dbutils.secrets.get(scope="dear-keyvault-scope", key="dearsql-database")
username = dbutils.secrets.get(scope="dear-keyvault-scope", key="dearsql-username")
password = dbutils.secrets.get(scope="dear-keyvault-scope", key="dearsql-password")


# Establish ODBC connection
try:
    conn = pyodbc.connect(
        "Driver={ODBC Driver 18 for SQL Server};"
        "Server=tcp:dearsqldbserver.database.windows.net,1433;"
        "Database=dearsqldb;"
        "Uid=dearsqldbadmin;"
        f"Pwd={password};"
        "Encrypt=yes;"
        "TrustServerCertificate=no;"
        "Connection Timeout=260;"
    )
    cursor = conn.cursor()
    logger.info("Connected to Azure SQL Database successfully")

    # Check and add new columns dynamically
    for key in metadata.keys():
        cursor.execute(f"""
            IF NOT EXISTS (
                SELECT * FROM INFORMATION_SCHEMA.COLUMNS 
                WHERE TABLE_NAME = 'ImageMetadata' AND COLUMN_NAME = ?
            )
            BEGIN
                ALTER TABLE ImageMetadata ADD [{key}] NVARCHAR(MAX);
            END;
        """, key)
        logger.info(f"Checked/added column: {key}")
    
    # Prepare INSERT query dynamically based on metadata keys
    columns = ["ImageDataPath", "Inserted_Time"] + [f"[{key}]" for key in metadata.keys()]
    placeholders = ["?", "?"] + ["?" for _ in metadata.keys()]
    insert_query = f"INSERT INTO ImageMetadata ({', '.join(columns)}) VALUES ({', '.join(placeholders)})"

    # Insert data
    Inserted_Time = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
    values = [Image_data_path, Inserted_Time] + [str(metadata[key]) if metadata[key] is not None else None for key in metadata.keys()] 
    cursor.execute(insert_query, values)
    logger.info(f"Inserted data into ImageMetadata with imageDataPath: {Image_data_path} and inserted_at: {Inserted_Time}")

    # Commit changes
    conn.commit()
    cursor.close()
    conn.close()
    logger.info("Data inserted into Azure SQL Database successfully")

except Exception as e:
    logger.error(f"Error inserting data into Azure SQL Database: {e}")
    raise

# Exit notebook and return metadata
dbutils.notebook.exit(metadata)